In [2]:
import numpy as np
from scipy import linalg
from sklearn.metrics import mean_absolute_error
import tensorflow as tf
import scipy.io as scio

In [3]:
## Evaluation Metrics: MModality, Diversity, Jerk, Acceleration, Velocity, Average Position Error, Mean Absolute Error

def calculate_multimodality(activation, multimodality_times):
    assert len(activation.shape) == 3
    assert activation.shape[1] > multimodality_times
    num_per_sent = activation.shape[1]

    first_dices = np.random.choice(num_per_sent, multimodality_times, replace=False)
    second_dices = np.random.choice(num_per_sent, multimodality_times, replace=False)
    dist = linalg.norm(activation[:, first_dices] - activation[:, second_dices], axis=2)
    return dist.mean()

def calculate_diversity(activation, diversity_times):
    assert len(activation.shape) == 2
    assert activation.shape[0] > diversity_times
    num_samples = activation.shape[0]

    first_indices = np.random.choice(num_samples, diversity_times, replace=False)
    second_indices = np.random.choice(num_samples, diversity_times, replace=False)
    dist = linalg.norm(activation[first_indices] - activation[second_indices], axis=1)
    return dist.mean()

def compute_jerks(data, dim=3):
    """Compute jerk between adjacent frames

      Args:
          data:         array containing joint positions of gesture
          dim:          gesture dimensionality

      Returns:
          np.ndarray:   jerks of each joint averaged over all frames
    """

    # Third derivative of position is jerk
    jerks = np.diff(data, n=3, axis=0)

    num_jerks = jerks.shape[0]
    num_joints = jerks.shape[1] // dim

    jerk_norms = np.zeros((num_jerks, num_joints))

    for i in range(num_jerks):
        for j in range(num_joints):
            x1 = j * dim + 0
            x2 = j * dim + dim
            jerk_norms[i, j] = np.linalg.norm(jerks[i, x1:x2])

    average = np.mean(jerk_norms, axis=0)

    # Take into account that frame rate was 10 fps
    scaled_av = average * 10 * 10 * 10

    return scaled_av

def compute_velocity(data, dim=3):
    """Compute velocity between adjacent frames

      Args:
          data:         array containing joint positions of gesture
          dim:          gesture dimensionality

      Returns:
          vel_norms:    velocities of each joint between each adjacent frame
    """

    # Flatten the array of 3d coords
    coords = data.reshape(data.shape[0], -1)

    # First derivative of position is velocity
    vels = np.diff(coords, n=1, axis=0)

    num_vels = vels.shape[0]
    num_joints = vels.shape[1] // dim

    vel_norms = np.zeros((num_vels, num_joints))

    for i in range(num_vels):
        for j in range(num_joints):
            x1 = j * dim + 0
            x2 = j * dim + dim
            vel_norms[i, j] = np.linalg.norm(vels[i, x1:x2])

    return vel_norms * 10

def compute_acceleration(data, dim=3):
    """Compute acceleration between adjacent frames

      Args:
          data:         array containing joint positions of gesture
          dim:          gesture dimensionality

      Returns:
          np.ndarray:   accelerations of each joint averaged over all frames
    """

    # Second derivative of position is acceleration
    accs = np.diff(data, n=2, axis=0)

    num_accs = accs.shape[0]
    num_joints = accs.shape[1] // dim

    acc_norms = np.zeros((num_accs, num_joints))

    for i in range(num_accs):
        for j in range(num_joints):
            x1 = j * dim + 0
            x2 = j * dim + dim
            acc_norms[i, j] = np.linalg.norm(accs[i, x1:x2])

    average = np.mean(acc_norms, axis=0)

    # Take into account that frame rate was 10 fps
    scaled_av = average * 10 * 10

    return scaled_av

def APE(original, predicted, dim=3):
    """Compute Average Position Error (APE)

      Args:
          original:     array containing joint positions of original gesture
          predicted:    array containing joint positions of predicted gesture
          dim:          gesture dimensionality

      Returns:
          np.ndarray:   APE between original and predicted for each joint
    """

    num_frames = predicted.shape[0]
    num_joints = predicted.shape[1] // dim

    diffs = np.zeros((num_frames, num_joints))

    for i in range(num_frames):
        for j in range(num_joints):
            x1 = j * dim + 0
            x2 = j * dim + dim
            diffs[i, j] = np.linalg.norm(
                original[i, x1:x2] - predicted[i, x1:x2])

    return np.mean(diffs, axis=0)

def MAE(ground_truth, pred_motion):
    return mean_absolute_error(ground_truth, pred_motion)

In [23]:
case_number = 18
i = 150
'''
#For validation set
fake_motion = np.load(f'../Hyperparameter_Tuning_and_Ablation_Study/Test_and_Generated_Data/Case_{case_number}/Generated_Data/generated_output_epoch_{i}.npy') 
real_motion = np.load(f'../Hyperparameter_Tuning_and_Ablation_Study/Test_and_Generated_Data/Case_{case_number}/Test_Data/test_action_batch_epoch_{i}.npy')
'''

# For test set
fake_test_motion = np.load('../Hyperparameter_Tuning_and_Ablation_Study/Model/Case_18/LSTM_Test_Motion_Model_Epoch_150.npy')
test_motion = np.load('../Data/test_action.npy')


In [24]:
# Compute Jerk, Acceleration, Velocity for real motion and fake motion

# Compute average jerk
real_jerks = compute_jerks(test_motion)
fake_jerks = compute_jerks(fake_test_motion)

# Compute average acceleration
real_acc = compute_acceleration(test_motion)
fake_acc = compute_acceleration(fake_test_motion)

# Compute average velocity
real_vel = compute_velocity(test_motion)
fake_vel = compute_velocity(fake_test_motion)

# Compute APE and MAE
mae = MAE(test_motion[2],fake_test_motion[2])
ape = APE(test_motion,fake_test_motion)

'''
print("****Real Motion****\n")
print(f"Mean of Jerks: {np.mean(real_jerks)}\n")
print(f"Standard Deviation of Jerks: {np.std(real_jerks)}\n")
print(f"Mean Acceleration: {np.mean(real_acc)}\n")
print(f"Standard Deviation of Acceleration: {np.std(real_acc)}\n")
print(f"Mean Velocity: {np.mean(real_vel)}\n")
print(f"Standard Deviation of Velocity: {np.std(real_vel)}\n")
'''

print("****Fake motion****\n")
print(f"Mean of Jerks: {np.mean(fake_jerks)}\n")
print(f"Standard Deviation of Jerks: {np.std(fake_jerks)}\n")
print(f"Mean Acceleration: {np.mean(fake_acc)}\n")
print(f"Standard Deviation of Acceleration: {np.std(fake_acc)}\n")
print(f"Mean Velocity: {np.mean(fake_vel)}\n")
print(f"Standard Deviation of Velocity: {np.std(fake_vel)}\n")

print(f"MAE: {mae}\n")
print(f"Mean APE: {np.mean(ape)}\n")
print(f"Standard Deviation APE: {np.std(ape)}\n")

****Fake motion****

Mean of Jerks: 5080.349920860057

Standard Deviation of Jerks: 170.9912367991335

Mean Acceleration: 277.6417024379138

Standard Deviation of Acceleration: 9.289599624518313

Mean Velocity: 2.769688226065122

Standard Deviation of Velocity: 2.0947496534602763

MAE: 0.07558558136224747

Mean APE: 1.371185125800744

Standard Deviation APE: 0.017058583466037343



In [31]:
# Evaluate using Diversity

fake_test_motion = np.load('../Hyperparameter_Tuning_and_Ablation_Study/Model/Case_18/poses_features.npy')

print(fake_test_motion.shape)

diversity = calculate_diversity(fake_test_motion, 500)
print(f'Diversity: {diversity}')

(4789, 32)
Diversity: 11.230298042297363


In [6]:
# Evaluate using MModality
case_number = 20

MModality_motion = np.load(f'../Hyperparameter_Tuning_and_Ablation_Study/Model/Case_18/mmodality_motion.npy')

print(MModality_motion.shape)


multimodality = calculate_multimodality(MModality_motion, 10)
print(f'Diversity: {multimodality}')


(50, 20, 32)
Diversity: 21.419673919677734
